In [1]:
# import pandas as pd
# import numpy as np
# import ast
# import re
# from sklearn.metrics.pairwise import cosine_similarity
# from tqdm import tqdm
# tqdm.pandas()

In [2]:
# open_path = './data/0f. vector_date_data_.csv'
# vector_df = pd.read_csv(open_path)

In [3]:
# .csv로 읽었을때 사용
# def preprocess_vector_string(vector_string):
#     try:
#         # 벡터 문자열 양 끝 공백 제거
#         vector_string = vector_string.strip()

#         # 실수 형식의 값만 추출 (양수, 음수, 소수점, 지수 표현 모두 포함)
#         numbers = re.findall(r'-?\d+\.?\d*(?:e[+-]?\d+)?', vector_string)
        
#         # 추출된 숫자 리스트를 쉼표로 구분된 문자열로 변환 (숫자 사이에 쉼표를 넣음)
#         clean_vector_string = ', '.join(numbers)
        
#         # 변환된 문자열을 numpy 배열로 변환
#         return np.array(ast.literal_eval(f'[{clean_vector_string}]'))
#     except Exception as e:
#         print(f"Error processing vector: {vector_string}, Error: {e}")
#         # 오류 발생 시 0으로 채워진 768차원 벡터 반환
#         return np.zeros(768)

In [4]:
# vector_df['vector'] = vector_df['vector'].progress_apply(preprocess_vector_string)

100%|██████████| 262695/262695 [11:09<00:00, 392.30it/s]


In [6]:
# save_path = './data/0f. vector_date_data_pre.csv'
# vector_df.to_pickle(save_path)

In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.metrics.pairwise import cosine_similarity
import datetime
from tqdm import tqdm
tqdm.pandas()


In [2]:
open_path = './data/0f. vector_date_data_pre.csv'
vector_df = pd.read_pickle(open_path)

In [3]:
# 월 필터링
vector_df['publication_date'] = pd.to_datetime(vector_df['publication_date'])
month_df = vector_df[(vector_df['publication_date'].dt.year == 2024) &
                            (vector_df['publication_date'].dt.month == 7)]

In [4]:
vector_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262695 entries, 0 to 262694
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   tag               262695 non-null  object        
 1   vector            262695 non-null  object        
 2   article_url       262695 non-null  object        
 3   publication_date  262695 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 8.0+ MB


In [5]:
month_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 90334 entries, 0 to 90333
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   tag               90334 non-null  object        
 1   vector            90334 non-null  object        
 2   article_url       90334 non-null  object        
 3   publication_date  90334 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 3.4+ MB


In [6]:
vector_df['site'] = vector_df['article_url'].progress_apply(lambda url: 'naver' if 'naver' in url else ('daum' if 'daum' in url else 'other'))
vector_df['sector'] = vector_df['site'].progress_apply(lambda site: 'IT' if site == 'naver' else ('Economy' if site == 'daum' else 'Empty'))

100%|██████████| 262695/262695 [00:00<00:00, 1799085.11it/s]


In [7]:
vector_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262695 entries, 0 to 262694
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   tag               262695 non-null  object        
 1   vector            262695 non-null  object        
 2   article_url       262695 non-null  object        
 3   publication_date  262695 non-null  datetime64[ns]
 4   site              262695 non-null  object        
 5   sector            262695 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 12.0+ MB


In [ ]:
# gpu 메세지 표시
# 현재 3개월치 중에 1/3 만 처리 적용
def to_gpu(tensor):
    return tensor.cuda()

# 처리할 샘플의 개수 설정
total_samples = len(vector_df['vector'][:87565])
count = 0   # 카운트
step = 100  # 몇 개마다 진행 상황을 표시할지 설정

results = []

# 벡터들을 GPU로 전송
vectors = torch.tensor(np.stack(vector_df['vector'].values)).cuda()

# 모든 벡터에 대해 반복
for i in range(total_samples):
    sample_vector = vectors[i].unsqueeze(0)  # 차원을 맞추기 위해 unsqueeze(0) 적용

    similarity_scores = torch.cosine_similarity(sample_vector, vectors, dim=1)

    # 자기 자신 1입력
    similarity_scores[i] = -1

    # CPU로 데이터 변환 (필요한 경우)
    similarity_scores = similarity_scores.cpu().numpy()

    # 'naver'가 포함된 사이트 필터링 및 상위 5개 선택
    naver_sim = pd.Series(similarity_scores, index=vector_df['article_url'])
    n_sim_dict = naver_sim[naver_sim.index.str.contains('naver')].nlargest(5).to_dict()

    # 'daum'이 포함된 사이트 필터링 및 상위 5개 선택
    daum_sim = pd.Series(similarity_scores, index=vector_df['article_url'])
    d_sim_dict = daum_sim[daum_sim.index.str.contains('daum')].nlargest(5).to_dict()

    # 각 벡터에 대한 결과 저장
    results.append({
        'article_url': vector_df['article_url'].iloc[i],
        'site': vector_df['site'].iloc[i],
        'sector': vector_df['sector'].iloc[i],
        'publication_date': vector_df['publication_date'].iloc[i],
        'n_sim': n_sim_dict,
        'd_sim': d_sim_dict
    })

    # 카운트 증가
    count += 1
    if count % step == 0:
        print(f"{count}/{total_samples} 작업 완료")

# 결과를 DataFrame으로 변환
final_df = pd.DataFrame(results)

# 결과 확인
print(final_df.head())


In [7]:
final_df.to_csv('./data/1f. simility_data_3m-1.csv')